<a href="https://colab.research.google.com/github/GenoKiller777/Hadoop/blob/main/SparkHadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving AirportCodeLocationLookupClean.csv to AirportCodeLocationLookupClean.csv
Saving FlightDelaysWithAirportCodes.csv to FlightDelaysWithAirportCodes.csv
Saving FlightWeatherWithAirportCode.csv to FlightWeatherWithAirportCode.csv
User uploaded file "AirportCodeLocationLookupClean.csv" with length 328194 bytes
User uploaded file "FlightDelaysWithAirportCodes.csv" with length 407244623 bytes
User uploaded file "FlightWeatherWithAirportCode.csv" with length 62556200 bytes


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=16a871f33c22d1bbcdf993382bfdae45774ab00c7b67c0b5522a368c0449b58e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [86]:
#import sparksession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType

from pyspark.sql.functions import col,isnan, when, count

In [4]:
#create sparksession object and provide appName
spark=SparkSession.builder.appName("Prueba_Local").getOrCreate()

In [5]:
#print spark version
print("Apache Spark version: ", spark.version)

Apache Spark version:  3.4.1


## Tablas

### _Tabla_ **AirportCodeLocationLookupClean**

In [22]:
schema_AirCode = StructType() \
.add("AIRPORT_ID",IntegerType(),True) \
.add("AIRPORT",StringType(),True) \
.add("DISPLAY_AIRPORT_NAM",StringType(),True) \
.add("LATITUDE",DoubleType(),True) \
.add("LONGITUDE",DoubleType(),True)

In [24]:
df_AirCode = spark.read.format("csv") \
.options(header='True', delimiter=',') \
.schema(schema_AirCode) \
.load("/content/AirportCodeLocationLookupClean.csv")

In [25]:
df_AirCode.printSchema()

root
 |-- AIRPORT_ID: integer (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- DISPLAY_AIRPORT_NAM: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



In [ ]:
df_AirCode.show(truncate=False)

### _Tabla_ **FlightDelaysWithAirportCodes**

In [92]:
schema_FligtDelays = StructType() \
.add("Year",IntegerType(),True) \
.add("Month",IntegerType(),True) \
.add("DayofMonth",IntegerType(),True) \
.add("DayOfWeek",IntegerType(),True) \
.add("Carrier",StringType(),True) \
.add("CRSDepTime",IntegerType(),True) \
.add("DepDelay",DoubleType(),True) \
.add("DepDel15",DoubleType(),False) \
.add("CRSArrTime",IntegerType(),True) \
.add("ArrDelay",DoubleType(),True) \
.add("ArrDel15",DoubleType(),True) \
.add("Cancelled",DoubleType(),True) \
.add("OriginAirportCode",StringType(),True) \
.add("OriginAirportName",StringType(),True) \
.add("OriginLatitude",DoubleType(),True) \
.add("OriginLongitude",DoubleType(),True) \
.add("DestAirportCode",StringType(),True) \
.add("DestAirportName",StringType(),True) \
.add("DestLatitude",DoubleType(),True) \
.add("DestLongitude",DoubleType(),True)

In [93]:
df_FligtDelays = spark.read.format("csv") \
.options(header='True', delimiter=',') \
.schema(schema_FligtDelays) \
.load("/content/FlightDelaysWithAirportCodes.csv")

In [94]:
df_FligtDelays.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DepDel15: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- ArrDel15: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- OriginAirportCode: string (nullable = true)
 |-- OriginAirportName: string (nullable = true)
 |-- OriginLatitude: double (nullable = true)
 |-- OriginLongitude: double (nullable = true)
 |-- DestAirportCode: string (nullable = true)
 |-- DestAirportName: string (nullable = true)
 |-- DestLatitude: double (nullable = true)
 |-- DestLongitude: double (nullable = true)



### _Tabla_ **FlightWeatherWithAirportCode**

In [39]:
schema_FlightWeather = StructType() \
.add("Year",IntegerType(),True) \
.add("Month",IntegerType(),True) \
.add("Day",IntegerType(),True) \
.add("Time",IntegerType(),True) \
.add("TimeZone",DoubleType(),True) \
.add("SkyCondition",StringType(),True) \
.add("Visibility",DoubleType(),True) \
.add("WeatherType",StringType(),True) \
.add("DryBulbFarenheit",DoubleType(),True) \
.add("DryBulbCelsius",DoubleType(),True) \
.add("WetBulbFarenheit",DoubleType(),True) \
.add("WetBulbCelsius",BooleanType(),True) \
.add("DewPointFarenheit",DoubleType(),True) \
.add("DewPointCelsius",DoubleType(),True) \
.add("RelativeHumidity",DoubleType(),True) \
.add("WindSpeed",DoubleType(),True) \
.add("WindDirection",DoubleType(),True) \
.add("ValueForWindCharacter",DoubleType(),True) \
.add("StationPressure",DoubleType(),True) \
.add("PressureTendency",DoubleType(),True) \
.add("PressureChange",DoubleType(),True) \
.add("SeaLevelPressure",DoubleType(),True) \
.add("RecordType",StringType(),True) \
.add("HourlyPrecip",StringType(),True) \
.add("Altimeter",DoubleType(),True) \
.add("AirportCode",StringType(),True) \
.add("DISPLAY_AIRPORT_NAME",StringType(),True) \
.add("LATITUDE",DoubleType(),True) \
.add("LONGITUDE",DoubleType(),True)

In [40]:
df_FlightWeather = spark.read.format("csv") \
.options(header='True', delimiter=',') \
.schema(schema_FlightWeather) \
.load("/content/FlightWeatherWithAirportCode.csv")

In [41]:
df_FlightWeather.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Time: integer (nullable = true)
 |-- TimeZone: double (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbCelsius: boolean (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- RelativeHumidity: double (nullable = true)
 |-- WindSpeed: double (nullable = true)
 |-- WindDirection: double (nullable = true)
 |-- ValueForWindCharacter: double (nullable = true)
 |-- StationPressure: double (nullable = true)
 |-- PressureTendency: double (nullable = true)
 |-- PressureChange: double (nullable = true)
 |-- SeaLevelPressure: double (nullable = true)
 |-- RecordType: 

## Cálculos

In [95]:
df_FligtDelays.show(10,truncate=False)

+----+-----+----------+---------+-------+----------+--------+--------+----------+--------+--------+---------+-----------------+------------------------------------------+--------------+---------------+---------------+----------------------------------------+------------+-------------+
|Year|Month|DayofMonth|DayOfWeek|Carrier|CRSDepTime|DepDelay|DepDel15|CRSArrTime|ArrDelay|ArrDel15|Cancelled|OriginAirportCode|OriginAirportName                         |OriginLatitude|OriginLongitude|DestAirportCode|DestAirportName                         |DestLatitude|DestLongitude|
+----+-----+----------+---------+-------+----------+--------+--------+----------+--------+--------+---------+-----------------+------------------------------------------+--------------+---------------+---------------+----------------------------------------+------------+-------------+
|2013|4    |19        |5        |DL     |837       |-3.0    |0.0     |1138      |1.0     |0.0     |0.0      |DTW              |Detroit Metro W

In [96]:
rows = df_FligtDelays.count()
print(f"Recuento de filas del DataFrame FlightDelaysWithAirportCodes: {rows}")

Recuento de filas del DataFrame FlightDelaysWithAirportCodes: 2719418


In [97]:
null_DepDel = (df_FligtDelays.filter(df_FligtDelays.DepDel15.isNull()).count())
print(f"Recuento de valores nulos del campo DepDel15 del DataFrame FlightDelaysWithAirportCodes : {null_DepDel}")

Recuento de valores nulos del campo DepDel15 del DataFrame FlightDelaysWithAirportCodes : 27444


### Verificación que se hayan eliminado las filas con datos faltantes para la columna **DepDel15**

In [106]:
verif_rows = df_FligtDelays.filter(~df_FligtDelays.DepDel15.contains('None') & \
                ~df_FligtDelays.DepDel15.contains('NULL') & \
                ~isnan(df_FligtDelays.DepDel15) & \
                ~df_FligtDelays.DepDel15.isNull()).count()


In [109]:
print(f"Recuento de valores, luego de restar los nulos del campo DepDel15 del DataFrame FlightDelaysWithAirportCodes : {verif_rows}")

Recuento de valores, luego de restar los nulos del campo DepDel15 del DataFrame FlightDelaysWithAirportCodes : 2691974


### Preparar los datos meteorológicos

In [110]:
df_FlightWeather.show(10,truncate=False)

+----+-----+---+----+--------+--------------------+----------+-----------+----------------+--------------+----------------+--------------+-----------------+---------------+----------------+---------+-------------+---------------------+---------------+----------------+--------------+----------------+----------+------------+---------+-----------+------------------------------+-----------+------------+
|Year|Month|Day|Time|TimeZone|SkyCondition        |Visibility|WeatherType|DryBulbFarenheit|DryBulbCelsius|WetBulbFarenheit|WetBulbCelsius|DewPointFarenheit|DewPointCelsius|RelativeHumidity|WindSpeed|WindDirection|ValueForWindCharacter|StationPressure|PressureTendency|PressureChange|SeaLevelPressure|RecordType|HourlyPrecip|Altimeter|AirportCode|DISPLAY_AIRPORT_NAME          |LATITUDE   |LONGITUDE   |
+----+-----+---+----+--------+--------------------+----------+-----------+----------------+--------------+----------------+--------------+-----------------+---------------+----------------+-----

In [ ]:
df_FlightWeather.createOrReplaceTempView("flight_weather_with_airport_code")

In [111]:
rows_weather = df_FlightWeather.count()
print(f"Recuento de filas del DataFrame FlightWeatherWithAirportCode: {rows_weather}")

Recuento de filas del DataFrame FlightWeatherWithAirportCode: 406516


In [115]:
spark.sql('select count(*) from flight_weather_with_airport_code').show()

+--------+
|count(1)|
+--------+
|  406516|
+--------+



## _No me salio la parte de R_

### WindSpeed

In [121]:
from pyspark.sql.functions import countDistinct

df_distcount_weather = df_FlightWeather.select(countDistinct("WindSpeed"))
df_distcount_weather.show()

+-------------------------+
|count(DISTINCT WindSpeed)|
+-------------------------+
|                       45|
+-------------------------+



In [117]:
spark.sql('select distinct WindSpeed from flight_weather_with_airport_code').show(45)

+---------+
|WindSpeed|
+---------+
|      8.0|
|      0.0|
|      7.0|
|     29.0|
|     47.0|
|     44.0|
|     null|
|     62.0|
|     18.0|
|     39.0|
|     34.0|
|     37.0|
|     25.0|
|     36.0|
|     41.0|
|     23.0|
|     45.0|
|     31.0|
|     11.0|
|     21.0|
|     51.0|
|     14.0|
|     22.0|
|      3.0|
|     19.0|
|     53.0|
|     46.0|
|     28.0|
|     17.0|
|     38.0|
|     10.0|
|     30.0|
|     40.0|
|     13.0|
|     33.0|
|      6.0|
|     20.0|
|     32.0|
|     15.0|
|      5.0|
|     24.0|
|      9.0|
|     26.0|
|     16.0|
|     43.0|
+---------+
only showing top 45 rows



### SeaLevelPressure

In [120]:
df_distcount_SeaLevelPressure = df_FlightWeather.select(countDistinct("SeaLevelPressure"))
df_distcount_SeaLevelPressure.show()

+--------------------------------+
|count(DISTINCT SeaLevelPressure)|
+--------------------------------+
|                             182|
+--------------------------------+



In [122]:
spark.sql('select distinct SeaLevelPressure from flight_weather_with_airport_code').show()

+----------------+
|SeaLevelPressure|
+----------------+
|           30.49|
|           29.47|
|           29.74|
|           29.42|
|           29.93|
|           30.73|
|           29.77|
|           29.83|
|           30.28|
|           29.36|
|           29.84|
|           30.56|
|           29.78|
|           29.37|
|           29.69|
|           29.97|
|           30.46|
|           30.58|
|           30.68|
|           30.76|
+----------------+
only showing top 20 rows



### HourlyPrecip

In [123]:
spark.sql('select distinct HourlyPrecip from flight_weather_with_airport_code').show()

+------------+
|HourlyPrecip|
+------------+
|        0.55|
|        0.07|
|        0.75|
|        0.59|
|        1.53|
|        2.49|
|        0.32|
|        0.03|
|        0.11|
|        1.23|
|        1.68|
|        1.41|
|        0.60|
|        1.85|
|        0.31|
|        0.73|
|        0.14|
|        0.74|
|        0.78|
|        0.57|
+------------+
only showing top 20 rows



### Limpiar datos meteorológicos - preparación del DataSet

In [124]:
dfWeather = spark.sql("select AirportCode, cast(Month as int) Month, cast(Day as int) Day, cast(Time as int) Time, WindSpeed, SeaLevelPressure, HourlyPrecip from flight_weather_with_airport_code")

In [125]:
dfWeather.show()

+-----------+-----+---+----+---------+----------------+------------+
|AirportCode|Month|Day|Time|WindSpeed|SeaLevelPressure|HourlyPrecip|
+-----------+-----+---+----+---------+----------------+------------+
|        SJU|    4|  1|  56|     13.0|           30.06|           T|
|        SJU|    4|  1| 156|     10.0|           30.05|        null|
|        SJU|    4|  1| 256|      9.0|           30.03|        null|
|        SJU|    4|  1| 356|      9.0|           30.03|        null|
|        SJU|    4|  1| 456|      7.0|           30.04|        null|
|        SJU|    4|  1| 556|      7.0|           30.05|        null|
|        SJU|    4|  1| 656|      9.0|           30.07|        null|
|        SJU|    4|  1| 756|     13.0|            30.1|        null|
|        SJU|    4|  1| 856|     14.0|           30.11|        null|
|        SJU|    4|  1| 956|     16.0|           30.12|        null|
|        SJU|    4|  1|1056|     17.0|           30.12|        null|
|        SJU|    4|  1|1156|     1

In [126]:
pprint.pprint(dfWeather.dtypes)

[('AirportCode', 'string'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Time', 'int'),
 ('WindSpeed', 'double'),
 ('SeaLevelPressure', 'double'),
 ('HourlyPrecip', 'string')]


In [143]:
# Round Time down to the next hour, since that is the hour for which we want to use flight data. Then, add the rounded Time to a new column named "Hour", and append that column to the dfWeather DataFrame.
df = dfWeather.withColumn('Hour', F.floor(dfWeather['Time']/100))

# Replace any missing HourlyPrecip and WindSpeed values with 0.0
df = df.fillna('0.0', subset=['HourlyPrecip', 'WindSpeed'])

# Replace any WindSpeed values of "M" with 0.005
df = df.replace('M', '0.005', 'WindSpeed')

# Replace any SeaLevelPressure values of "M" with 29.92 (the average pressure)
df = df.replace('M', '29.92', 'SeaLevelPressure')

# Replace any HourlyPrecip values of "T" (trace) with 0.005
df = df.replace('T', '0.005', 'HourlyPrecip')

# Be sure to convert WindSpeed, SeaLevelPressure, and HourlyPrecip columns to float
# Define a new DataFrame that includes just the columns being used by the model, including the new Hour feature
dfWeather_clean = df.select('AirportCode', 'Month', 'Day', 'Hour', df['WindSpeed'].cast('float'), df['SeaLevelPressure'].cast('float'), df['HourlyPrecip'].cast('float'))

In [144]:
display(dfWeather_clean)

DataFrame[AirportCode: string, Month: int, Day: int, Hour: bigint, WindSpeed: float, SeaLevelPressure: float, HourlyPrecip: float]

In [145]:
pprint.pprint(dfWeather_clean.dtypes)

[('AirportCode', 'string'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('Hour', 'bigint'),
 ('WindSpeed', 'float'),
 ('SeaLevelPressure', 'float'),
 ('HourlyPrecip', 'float')]


In [146]:
dfWeather_clean.write.mode("overwrite").saveAsTable("flight_weather_clean")

In [147]:
dfWeather_clean.select("*").count()

406516